In [91]:
import os

import pandas as pd
import pandas_ta as ta
import numpy as np
import matplotlib.pyplot as plt
import requests
from datetime import datetime, timedelta
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from getpass import getpass

In [26]:
time_to = int(datetime.now().timestamp())
time_from = int((time_to - timedelta(days=120).total_seconds()))
time_from, time_to

(1701893577, 1712261577)

In [24]:
resp = requests.get('https://api.kraken.com/0/public/OHLC?pair=WIFUSD&interval=60&since=1701891170')

data = resp.json()['result']['WIFUSD']


In [27]:
pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'vwap', 'volume', 'count']).tail()

,time,open,high,low,close,vwap,volume,count
715,1712246400,3.8263,3.8786,3.7837,3.8495,3.7964,17854.04157,70
716,1712250000,3.8165,3.9251,3.7979,3.9104,3.8438,25927.91534,99
717,1712253600,3.9109,3.9602,3.7277,3.7436,3.8499,58290.87371,263
718,1712257200,3.7274,3.7439,3.6621,3.6621,3.7086,70596.96143,286
719,1712260800,3.6608,3.6608,3.6255,3.6255,3.6384,6372.34409,35


In [35]:
def read_ohlcv_date(symbol: str) -> pd.DataFrame:
    time_to = int(datetime.now().timestamp())
    time_from = int((time_to - timedelta(days=120).total_seconds()))
    resp = requests.get(f'https://api.kraken.com/0/public/OHLC?pair={symbol.upper()}USD&interval=60&since={time_from}')
    data = resp.json()['result'][f'{symbol.upper()}USD']
    df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'vwap', 'volume', 'count'])
    df.drop(columns=['vwap', 'count'], inplace=True)
    df.rename(columns={'time': 'date'}, inplace=True)
    df['date'] = pd.to_datetime(df['date'], unit='s')
    df['open'] = df['open'].astype(float)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    df['close'] = df['close'].astype(float)
    df['volume'] = df['volume'].astype(float)
    df.set_index('date', inplace=True)
    df = df.iloc[:-1]
    return df

In [40]:
sol_df = read_ohlcv_date('sol')
sol_df.tail()

,open,high,low,close,volume
date,,,,,
2024-04-04 15:00:00,188.31,188.38,186.49,186.98,10802.083933
2024-04-04 16:00:00,186.99,188.39,186.93,187.40,11567.076114
2024-04-04 17:00:00,187.41,189.25,187.02,189.13,5752.176989
2024-04-04 18:00:00,189.13,190.07,185.71,185.99,44998.024704
2024-04-04 19:00:00,185.98,187.00,185.66,185.71,17434.080800


In [41]:
sol_df.dtypes

open      float64
high      float64
low       float64
close     float64
volume    float64
dtype: object

In [42]:
sol_df.ta.macd(append=True)
sol_df.ta.rsi(append=True)
sol_df.ta.bbands(append=True)
sol_df.ta.obv(append=True)

sol_df.ta.sma(length=20, append=True)
sol_df.ta.ema(length=50, append=True)
sol_df.ta.stoch(append=True)
sol_df.ta.adx(append=True)

sol_df.ta.willr(append=True)
sol_df.ta.cmf(append=True)
sol_df.ta.psar(append=True)

,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2
date,,,,
2024-03-05 21:00:00,NaN,NaN,0.02,0
2024-03-05 22:00:00,NaN,123.87,0.02,1
2024-03-05 23:00:00,118.580000,NaN,0.02,1
2024-03-06 00:00:00,118.580000,NaN,0.02,0
2024-03-06 01:00:00,118.756400,NaN,0.02,0
...,...,...,...,...
2024-04-04 15:00:00,182.685913,NaN,0.08,0
2024-04-04 16:00:00,183.239840,NaN,0.08,0
2024-04-04 17:00:00,183.749453,NaN,0.08,0


In [43]:
sol_df.tail()

,open,high,low,close,volume,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,BBL_5_2.0,...,STOCHd_14_3_3,ADX_14,DMP_14,DMN_14,WILLR_14,CMF_20,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2
date,,,,,,,,,,,,,,,,,,,,,
2024-04-04 15:00:00,188.31,188.38,186.49,186.98,10802.083933,0.249549,0.445671,-0.196122,53.482132,185.198690,...,84.132082,15.031661,24.719187,18.330878,-28.463203,0.139405,182.685913,NaN,0.08,0
2024-04-04 16:00:00,186.99,188.39,186.93,187.40,11567.076114,0.318373,0.411596,-0.093223,54.917410,186.323149,...,82.271468,15.022850,23.552654,17.441166,-23.917749,0.157051,183.239840,NaN,0.08,0
2024-04-04 17:00:00,187.41,189.25,187.02,189.13,5752.176989,0.506672,0.479916,0.026756,60.344874,186.513669,...,79.990380,15.439275,24.662580,16.151669,-5.194805,0.090559,183.749453,NaN,0.08,0
2024-04-04 18:00:00,189.13,190.07,185.71,185.99,44998.024704,0.397942,0.296949,0.100993,48.849643,185.398630,...,77.450861,14.938403,21.340463,18.023256,-49.695493,-0.122071,184.218297,NaN,0.10,0
2024-04-04 19:00:00,185.98,187.00,185.66,185.71,17434.080800,0.285883,0.147912,0.137971,47.972078,184.613391,...,72.083712,14.440714,20.429627,17.413261,-59.972490,-0.158017,184.803467,NaN,0.10,0


In [45]:
sol_df['OBV_in_million'] = sol_df['OBV']/1e7
sol_df['MACD_histogram_12_26_9'] = sol_df['MACDh_12_26_9']

In [46]:
last_day_summary = sol_df.iloc[-1][['close',
    'MACD_12_26_9','MACD_histogram_12_26_9', 'RSI_14', 'BBL_5_2.0', 'BBM_5_2.0', 'BBU_5_2.0','SMA_20', 'EMA_50','OBV_in_million', 'STOCHk_14_3_3', 
    'STOCHd_14_3_3', 'ADX_14',  'WILLR_14', 'CMF_20', 
    'PSARl_0.02_0.2', 'PSARs_0.02_0.2'
]]

print("Summary of Technical Indicators for the Last Day:")
print(last_day_summary)

Summary of Technical Indicators for the Last Day:
close                     185.710000
MACD_12_26_9                0.285883
MACD_histogram_12_26_9      0.147912
RSI_14                     47.972078
BBL_5_2.0                 184.613391
BBM_5_2.0                 187.042000
BBU_5_2.0                 189.470609
SMA_20                    185.639000
EMA_50                    186.617856
OBV_in_million              0.079841
STOCHk_14_3_3              61.712404
STOCHd_14_3_3              72.083712
ADX_14                     14.440714
WILLR_14                  -59.972490
CMF_20                     -0.158017
PSARl_0.02_0.2            184.803467
PSARs_0.02_0.2                   NaN
Name: 2024-04-04 19:00:00, dtype: float64


In [52]:
OPENAI_API_KEY = getpass("Enter your OpenAI API Key: ")

In [55]:
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [115]:
llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0)
# llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [116]:
prompt = ChatPromptTemplate.from_template("""
Assume the role as a leading Technical Analysis (TA) expert in the stock market, \
a modern counterpart to Charles Dow, John Bollinger, and Alan Andrews. \
Your mastery encompasses both stock fundamentals and intricate technical indicators. \
You possess the ability to decode complex market dynamics, \
providing clear insights and recommendations backed by a thorough understanding of interrelated factors. \
Your expertise extends to practical tools like the pandas_ta module, \
allowing you to navigate data intricacies with ease. \
As a TA authority, your role is to decipher market trends, make informed predictions, and offer valuable perspectives.

given {symbol} TA data as below on the last trading day, what will be the next few days possible crypto price movement? 

Summary of Technical Indicators for the Last Day:
{last_day_summary}""".format(symbol="SOL", last_day_summary=last_day_summary))

In [117]:
chain = prompt | llm | StrOutputParser()

In [118]:
response = chain.invoke({})

In [119]:
print(response)

Analyzing the provided technical indicators for SOL on the last trading day, we can derive insights into the potential price movement in the next few days. Let's break down the key indicators:

1. **MACD (Moving Average Convergence Divergence)**: The MACD value is positive (0.285883), and the MACD histogram (0.147912) is also positive, indicating a bullish momentum. However, the relatively small magnitude of these values suggests that while the trend is upward, it might not be very strong.

2. **RSI (Relative Strength Index)**: The RSI stands at 47.972078, which is near the midpoint of 50. This indicates that the asset is neither overbought nor oversold, suggesting a potential continuation of the current trend or a consolidation phase.

3. **Bollinger Bands (BB)**: The closing price (185.710000) is slightly below the BB middle band (187.042000), indicating a slight bearish bias in the very short term. However, the price is within the bands, suggesting that volatility is not extreme.

4